# Azure AI Search med NVIDIA NIM och LlamaIndex-integration

I den här notebooken kommer vi att visa hur man använder NVIDIAs AI-modeller och LlamaIndex för att skapa en kraftfull Retrieval-Augmented Generation (RAG)-pipeline. Vi kommer att använda NVIDIAs LLMs och embeddings, integrera dem med Azure AI Search som vektorlagring och utföra RAG för att förbättra sökkvalitet och effektivitet.

## Fördelar
- **Skalbarhet**: Utnyttja NVIDIAs stora språkmodeller och Azure AI Search för skalbar och effektiv hämtning.
- **Kostnadseffektivitet**: Optimera sökning och hämtning med effektiv vektorlagring och hybridtekniker för sökning.
- **Hög prestanda**: Kombinera kraftfulla LLMs med vektoriserad sökning för snabbare och mer exakta svar.
- **Kvalitet**: Bibehåll hög sökkvalitet genom att grunda LLM-svar med relevanta hämtade dokument.

## Förutsättningar
- 🐍 Python 3.9 eller högre
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API-nyckel för åtkomst till NVIDIAs LLMs och Embeddings via NVIDIA NIM-mikrotjänster

## Funktioner som täcks
- ✅ NVIDIA LLM-integration (vi kommer att använda [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (vi kommer att använda [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Avancerade hämtlägen i Azure AI Search
- ✅ Dokumentindexering med LlamaIndex
- ✅ RAG med Azure AI Search och LlamaIndex tillsammans med NVIDIAs LLMs

Nu kör vi!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Installation och krav
Skapa en Python-miljö med Python version >3.10.

## Kom igång!


För att komma igång behöver du en `NVIDIA_API_KEY` för att använda NVIDIA AI Foundation-modeller:  
1) Skapa ett gratis konto hos [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Klicka på den modell du vill använda.  
3) Under Input, välj fliken Python och klicka på **Get API Key** och sedan på **Generate Key**.  
4) Kopiera och spara den genererade nyckeln som NVIDIA_API_KEY. Därefter bör du ha tillgång till endpoints.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG-exempel med LLM och inbäddning
### 1) Initiera LLM
`llama-index-llms-nvidia`, även känd som NVIDIAs LLM-koppling, gör det möjligt att ansluta till och generera från kompatibla modeller som finns tillgängliga i NVIDIAs API-katalog. Se här för en lista över modeller för chattkomplettering: https://build.nvidia.com/search?term=Text-to-Text

Här kommer vi att använda **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Initiera inbäddningen
`llama-index-embeddings-nvidia`, även känd som NVIDIAs Embeddings-anslutning, gör det möjligt att ansluta till och generera från kompatibla modeller som finns tillgängliga i NVIDIAs API-katalog. Vi valde `nvidia/nv-embedqa-e5-v5` som inbäddningsmodell. Se här för en lista över textinbäddningsmodeller: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Skapa en Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Skapa en frågemotor för att ställa frågor om dina data

Här är en fråga som använder ren vektorsökning i Azure AI Search och förankrar svaret till vår LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Här är en fråga som använder hybrid sökning i Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analys av vektorsökning
LLM-svaret fångar korrekt de viktigaste ekonomiska konsekvenserna som nämns i källtexten angående Rysslands aktiemarknad. Det anger specifikt att den ryska aktiemarknaden upplevde ett betydande fall, förlorade 40% av sitt värde, och att handeln stoppades på grund av den pågående situationen. Detta svar stämmer väl överens med informationen i källan, vilket visar att LLM korrekt identifierade och sammanfattade relevanta detaljer om aktiemarknadens påverkan som ett resultat av Rysslands agerande och de införda sanktionerna.

#### Kommentarer om källnoder
Källnoderna ger en detaljerad redogörelse för de ekonomiska konsekvenser som Ryssland mötte på grund av internationella sanktioner. Texten framhäver att den ryska aktiemarknaden förlorade 40% av sitt värde och att handeln stoppades. Dessutom nämns andra ekonomiska följder, såsom Rubelns devalvering och den bredare isoleringen av Rysslands ekonomi. LLM-svaret destillerade effektivt de kritiska punkterna från dessa noder, med fokus på aktiemarknadens påverkan enligt frågans begäran.


Nu ska vi titta på ett exempel där Hybrid Search inte ger ett välgrundat svar:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybrid Search: LLM-svaranalys
LLM-svaret i Hybrid Search-exemplet indikerar att den tillhandahållna kontexten inte specificerar det exakta datumet för Rysslands invasion av Ukraina. Detta svar antyder att LLM använder den information som finns i källdokumenten men erkänner avsaknaden av precisa detaljer i texten.

Svaret är korrekt i att identifiera att kontexten nämner händelser relaterade till Rysslands aggression men inte fastställer det specifika invasionsdatumet. Detta visar LLM:s förmåga att förstå den tillhandahållna informationen samtidigt som den känner igen luckor i innehållet. LLM uppmanar effektivt användaren att söka externa källor eller historiska dokument för det exakta datumet, vilket visar en nivå av försiktighet när informationen är ofullständig.

### Analys av källnoder
Källnoderna i Hybrid Search-exemplet innehåller utdrag från ett tal som diskuterar USA:s svar på Rysslands agerande i Ukraina. Dessa noder betonar den bredare geopolitiska påverkan och de åtgärder som USA och dess allierade har vidtagit som svar på invasionen, men de nämner inte det specifika invasionsdatumet. Detta stämmer överens med LLM-svaret, som korrekt identifierar att kontexten saknar den exakta datuminformationen.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid med omrankning: Analys av LLM-svar
I exemplet Hybrid med omrankning ger LLM-svaret ytterligare kontext genom att notera att händelsen inträffade sex dagar innan talet hölls. Detta visar att LLM kan dra slutsatser om invasionsdatumet baserat på tidpunkten för talet, även om det fortfarande krävs att veta det exakta datumet för talet för att vara helt korrekt.

Detta svar demonstrerar en förbättrad förmåga att använda kontextledtrådar för att ge ett mer informativt svar. Det framhäver fördelen med omrankning, där LLM kan få tillgång till och prioritera mer relevant information för att ge en närmare approximation av den önskade detaljen (dvs. invasionsdatumet).

### Analys av källnoder
Källnoderna i detta exempel inkluderar referenser till tidpunkten för Rysslands invasion, specifikt med nämnandet att den inträffade sex dagar innan talet. Även om det exakta datumet fortfarande inte uttryckligen anges, ger noderna en tidsmässig kontext som gör det möjligt för LLM att ge ett mer nyanserat svar. Inkluderingen av denna detalj visar hur omrankning kan förbättra LLM:s förmåga att extrahera och dra slutsatser från den givna kontexten, vilket resulterar i ett mer korrekt och informativt svar.


**Observera:**
I den här anteckningsboken använde vi NVIDIA NIM mikrotjänster från NVIDIA API-katalogen.  
Ovanstående API:er, `NVIDIA (llms)`, `NVIDIAEmbedding`, och [Azure AI Search Semantic Hybrid Retrieval (inbyggd omrankning)](https://learn.microsoft.com/azure/search/semantic-search-overview). Observera att ovanstående API:er också kan stödja självhostade mikrotjänster.

**Exempel:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
